In [2]:
import numpy as np

import mne
import nibabel

# TODO:
# 1. FIX .clim setting breakage
# 2. add mne_analyze_colormap as colormap
# 3. add brain behind STC

In [3]:
data_path = mne.datasets.sample.data_path()
subjects_dir = data_path + '/subjects'
subj_dir = subjects_dir + '/sample/surf/'
stc = mne.read_source_estimate(data_path + '/MEG/sample/sample_audvis-meg-eeg')
stc.crop(0.09, 0.09)
stc = stc.morph('sample', grade=None, smooth=10, subjects_dir=subjects_dir,
                subject_from='sample')

Morphing data...
    Left-hemisphere map read.
    Right-hemisphere map read.
    Left-hemisphere map read.
    Right-hemisphere map read.
    10 smooth iterations done.
    10 smooth iterations done.
[done]


In [4]:
azimuths = dict(lh=dict(lat=270, med=90), rh=dict(lat=90, med=270))
hemis = ['lh']
views = ['lat']
hemi_data = dict(lh=stc.data[:len(stc.vertices[0]), 0],
                 rh=stc.data[len(stc.vertices[0]):, 0])

In [5]:
for hi, hemi in enumerate(hemis):
    data = hemi_data[hemi]
    data /= 23
    surf = subj_dir + '%s.inflated' % hemi
    curv = subj_dir + '%s.curv' % hemi
    rr, tris = nibabel.freesurfer.read_geometry(surf)
    tris = tris.astype(np.uint32)
    assert len(data) == len(rr)
    curv = nibabel.freesurfer.read_morph_data(curv)
    curv = (curv > 0).astype(float)
    curv = (curv - 0.5) / 3 + 0.5
    curv = curv[:, np.newaxis] * [1, 1, 1]

In [6]:
rr.shape, tris.shape, curv.shape

((155407, 3), (310810, 3), (155407, 3))

In [ ]:
from pythreejs import Plain
import numpy as np
from IPython.display import display

In [ ]:
vertices = rr
faces = tris
vertexcolors = curv
geom = PlainBufferGeometry(vertices=vertices, faces=faces, colors=vertexcolors)
mesh = Mesh(geometry=geom, material=PhongMaterial(shininess=0,vertexColors='VertexColors'))
camera = PerspectiveCamera(position=[500, 500, 500], fov=20,
                           children=[DirectionalLight(color='#ffffff',
                                                      position=[-3, 5, 1],
                                                      intensity=0.5)])
scene = Scene(children=[mesh, AmbientLight(color='#dddddd')])

renderer = Renderer(camera=camera, background='black', background_opacity=1,
                    scene=scene, controls=[OrbitControls(controlling=camera)])


In [ ]:
display(renderer)


In [ ]:
norm = np.linalg.norm(vertices, axis=1)
norm /= norm.max()
def color(n):
    newcolor=curv.copy()
    newcolor[:,0] = np.abs(np.sin(norm*n))
    newcolor[:,2] = np.abs(np.cos(norm*n))
    geom.colors = newcolor

In [ ]:
color(5)

In [ ]:
from ipywidgets import FloatSlider, VBox
x = FloatSlider(min=0, max=12)
def slide(change):
    color(x.value)
x.observe(slide)
display(VBox([x, renderer]))

In [ ]:
camera.position